### chainerのインストール

In [1]:
# !sudo chmod 777 /usr/local/lib/python3.6/dist-packages/__pycache__/
# !pip3 install chainer
# !git clone https://github.com/chainer/chainer.git
# !export PYTHONPATH=chainer/examples/text_classification/

In [2]:
!install nets

install: missing destination file operand after 'nets'
Try 'install --help' for more information.


In [23]:
import chainer
import chainer.function as F
import chainer.links as L
import numpy
from chainer import training
from chainer.training import extensions
import nets
from nlp_utils import convert_seq, transform_to_array

# class Encoder(cahiner.Chain):
#     def __init__(self, w):
#         super(Encoder, self)

ModuleNotFoundError: No module named 'nets'